In [21]:
try:
  # We must install required packages if we are in Google Colab
  import google.colab
  %pip install roboticstoolbox-python>=1.0.2
except:
  # We are not in Google Colab
  # Apply custon style to notebook
  from IPython.core.display import HTML
  import pathlib
  styles_path = pathlib.Path(pathlib.Path().absolute(), "style", "style.css")
  styles = open(styles_path, "r").read()
  HTML(f"<style>{styles}</style>")

$\large{\text{Foundations of Kinematics and Algorithms in Robotics}} \\ \large{\text{Module 4: Differential Spatial Mathematics}}$

$\text{By Jesse Haviland and Peter Corke}$

<br>

### Contents

[1.0 Rate of Change of Orientation](#1)  
[2.0 Rate of Change of Pose](#2)  
[3.0 Transforming Spatial Velocities](#3)  

In [22]:
# We will do the imports required for this notebook here

# numpy provides import array and linear algebra utilities
import numpy as np

# the robotics toolbox provides robotics specific functionality
import roboticstoolbox as rtb

# spatial math provides objects for representing transformations
import spatialmath as sm
import spatialmath.base as smb

import spatialgeometry as sg

from swift import Swift

# The matrix exponential method
from scipy.linalg import expm, logm

# sympy provides symbolic mathematic operations 
import sympy as sym

# setting up sympy to print nicely
from IPython import display as ds
from sympy.physics.mechanics import init_vprinting
from sympy.physics.vector import vlatex
init_vprinting(use_latex='mathjax')

<br>

<a id='1'></a>
# 1.0 Rate of Change of Orientation
---

## 1.1 Derivative of a Rotation and Angular Velocity


Recall that a rotation matrix is defined as

$$
    {^a\mathbf{R}_b}(t) = e^{[{^a\hat{\eta}_b}(t)]_\times \ \theta(t)}
$$

where ${^a\hat{\eta}_b}(t)$ is the rotation axis of frame ${b}$ with respect to frame ${a}$ and $\theta(t)$ is the rotation angle. While both of these quantities are functions of time, at an instant in time, we will assume that the rotation axis is fixed. The derivative with respect to time is

\begin{align*}
    {^a\dot{\mathbf{R}}_b}(t) 
    &= [{^a\hat{\eta}_b}(t)]_\times \ \dot{\theta}(t) \ e^{[{^a\hat{\eta}_b}(t)]_\times \ \theta(t)} \\
    &= [{^a\hat{\eta}_b}(t)]_\times \ \dot{\theta}(t) \ {^a\mathbf{R}_b}(t)
\end{align*}




where we can introduce the angular velocity of frame ${b}$ with respect to frame ${a}$ as

$$
    {^a\omega_b}(t) = {^a\hat{\eta}_b}(t) \ \dot{\theta}(t)
$$

where you may see $\hat{\eta}(t)$ also written as $\hat{\omega}(t)$ in other texts, these are equivalent. The angular velocity vector represents a similar concept as the Euler vector, it is a vector

$$
    \omega = \begin{pmatrix} \omega_x \\ \omega_y \\ \omega_z \end{pmatrix}
$$

that defines the _instantaneous_ axis and rate of rotation, with $\hat{\eta}$ representing the axis and $\dot{\theta}$ representing the rate.

The derivative of a rotation matrix can then be written as

$$
    \dot{\mathbf{R}}(t) = [{\omega}(t)]_\times \ \mathbf{R}(t)
$$

and then rearranged

\begin{align*}
    \dot{\mathbf{R}}(t) \ \mathbf{R}(t)^\top &=  \ [{\omega}(t)]_\times \\
    \mathsf{V}_\times \Big(
        \dot{\mathbf{R}}(t) \ \mathbf{R}(t)^\top
    \Big)
    &= \ {\omega}(t)
\end{align*}





which shows how we can obtain the angular velocity from the derivative of a rotation matrix.

In [23]:
# A variable for time
t = sym.symbols('t')

theta = sym.symbols('theta')

# Make the angle a functions of time
thetat = sym.Function(theta)(t)

# Make our rotation axis
etahat = np.array([0, 0, 1])

# Create skew of eta
etahat_sk = smb.skew(etahat)

# Create a rotation matrix
R = sym.simplify(sym.Matrix(etahat_sk * thetat).exp())

# Display the result
ds.display(ds.Math(f"[\\mathbf{{\\eta}}]_\\times = {sym.latex(sym.Matrix(eta_sk))}"))
ds.display(ds.Math(f"\\mathbf{{R}} = {sym.latex(sym.Matrix(R))}"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In the next cell, we will take the derivative of $\mathbf{R}$ using symbolic differentiation from `sympy`. This is not a very tricky operation, it only involves taking the derivative of a few `sin` and `cos` functions

In [24]:
# Compute the derivative of the rotation matrix using sympy
Rdot = sym.simplify(R.diff(t))

# Display the result
ds.display(ds.Math(f"\\dot{{\\mathbf{{R}}}} = {vlatex(sym.Matrix(Rdot))}"))

<IPython.core.display.Math object>

In the next cell, we will attempt the same differentiation using the theory outlined above and show we get the same result

In [25]:
# Compute the derivative using our theory above
Rdot = sym.simplify(etahat_sk * sym.diff(thetat, t) * R)

# Display the result
ds.display(ds.Math(f"\\dot{{\\mathbf{{R}}}} = {vlatex(sym.Matrix(Rdot))}"))

<IPython.core.display.Math object>

Now we will calculate the angular velocity $\omega$

In [26]:
omega = sym.simplify(smb.vex(Rdot @ R.T))

# Display the result
ds.display(ds.Math(f"\\omega = {vlatex(omega)}"))

<IPython.core.display.Math object>

Which gives us a rather intuitive result. In our example, we had a rotation about the $z$-axis and when we took the derivative of $\mathbf{R}$, we assumed that at that instant in time the axis of rotation is fixed. Therefore, the derivative of the rotation will also be about the $z$-axis. Given that the angular velocity $\omega$ describes the instantaneous axis and rate of rotation, we can see that the angular velocity is about the $z$-axis and the rate of rotation is $\dot{\theta}$. For rotations not about a single axis, the result is not so simple to interpret -- you can try modifying the axis of rotation in the above example to see what happens to the result.

## 1.2 Derivative of a Moving Reference Frame

Consider now that angular velocity is measured in the moving frame $\{b\}$, for example, it is measured by gyroscope sensors onboard a moving vehicle. We know that

$$
    {^a\dot{\mathbf{R}}_b} = {^a\mathbf{R}_b} \ [{^b\omega_\mathbf{b}}]_\times \in \mathbb{R}^{3 \times 3}
$$


and we see that the order of the rotation matrix and the skew-symmetric matrix have been swapped.

## 1.3 Derivative of a Quaternion

The derivatives of a unit quaternion is given by

$$
    {^a\dot{\mathring{\mathbf{q}}}_b} = \frac{1}{2} \ {^a\check{\omega}_b} \circ {^a\mathring{\mathbf{q}}_b}
$$

or for a moving frame

$$
    {^a\dot{\mathring{\mathbf{q}}}_b} = \frac{1}{2} \ {^a\mathring{\mathbf{q}}_b} \circ {^b\check{\omega}_\mathbf{b}}
$$

where ${^a\dot{\mathring{\mathbf{q}}}_b} \in \mathbb{H}$ is a regular quaternion (not a unit quaternion), and $\check{\omega}$ is a pure quaternion formed from the angular velocity vector

$$
    \check{\omega} = \begin{pmatrix} 0 \\ \omega_x \\ \omega_y \\ \omega_z \end{pmatrix}
$$.

## Test Your Understanding

#### Question 1

Given a rotation matrix $\mathbf{R} = \mathbf{R}_x(\theta(t))$

* What is $\dot{\mathbf{R}}$?
* What is the angular velocity $\omega$?
* What is $\dot{\mathring{\mathbf{q}}}$

#### Question 2

Given a rotation matrix $\mathbf{R} = \mathbf{R}_y(30^\circ) \ \mathbf{R}_y(45^\circ) \ \mathbf{R}_x(\theta(t))$

* What is $\dot{\mathbf{R}}$?
* What is the angular velocity $\omega$?
* What is $\dot{\mathring{\mathbf{q}}}$

#### Question 3

What robotic applications can you think where the derivative of a rotation may be used?

<br>

<a id='2'></a>
# 2.0 Rate of Change of Pose
---

The derivative of pose can be determined by expressing pose as an $\mathbf{SE}(3)$ matrix

$$
    {^a\mathbf{T}_b}(t) = \begin{pmatrix} {^a\mathbf{R}_b} & {^a\mathbf{t}_b} \\ \mathbf{0}_{1 \times 3} & 1 \end{pmatrix} \in \mathbf{SE}(3)
$$

and taking the derivative with respect to time, then substituting our previous rotational derivative

\begin{align*}
    {^a\dot{\mathbf{T}}_b} &= \begin{pmatrix} {^a\dot{\mathbf{R}}_b} & {^a\dot{\mathbf{t}}_b} \\ \mathbf{0}_{1 \times 3} & 0 \end{pmatrix} \\
    &= \begin{pmatrix} [{^a\omega_b}]_\times  \ {^a\mathbf{R}_b} & {^a\dot{\mathbf{t}}_b} \\ \mathbf{0}_{1 \times 3} & 0 \end{pmatrix} \in \mathbb{R}^{4 \times 4}
\end{align*}

The rate of change can be described in terms of the current orientation ${^a\mathbf{R}_b}$ and _two_ velocities. The linear, or translational, velocity ${^a\mathbf{v}_b} = {^a\dot{\mathbf{t}}_b} \in \mathbf{R}^3$ is the velocity of the origin of {$b$} with respect to {$a$}. The angular velocity ${^a\omega_b} \in \mathbf{R}^3$ has already been introduced. We can combine these two velocity vectors to create the spatial velocity vector

$$
    {^a\mathbf{\nu}_b} = \begin{pmatrix} {^a\mathbf{v}_b} \\ {^a\omega_b} \end{pmatrix} \in \mathbf{R}^6
$$

which is the instantaneous velocity of the origin of {$b$} with respect to {$a$}.

Every point in the body has the same angular velocity, but the translational velocity of a point depends on its position within the body. It is common to place {$b$} at the body’s center of mass.

## Test Your Understanding

#### Question 1

Given a transformation matrix $\mathbf{T} = \mathbf{T}_{t_x}(d(t))$

* What is $\dot{\mathbf{T}}$?
* What is the spatial velocty $\nu$?
* What is the translational velocity $\mathbf{v}$?
* What is the angular velocity $\omega$?

#### Question 2

Given a transformation matrix $\mathbf{T} = \mathbf{T}_{R_y}(30^\circ) \ \mathbf{T}_{t_x}(d(t))$

* What is $\dot{\mathbf{T}}$?
* What is the spatial velocty $\nu$?
* What is the translational velocity $\mathbf{v}$?
* What is the angular velocity $\omega$?

#### Question 3

Given a transformation matrix $\mathbf{T} = \mathbf{T}_{t_z}(0.1) \ \mathbf{T}_{R_y}(30^\circ) \ \mathbf{T}_{t_x}(d(t))$

* What is $\dot{\mathbf{T}}$?
* What is the spatial velocty $\nu$?
* What is the translational velocity $\mathbf{v}$?
* What is the angular velocity $\omega$?

#### Question 4

Write a Python method that can calculate the derivatives of the transformations in Questions 1-3

#### Question 5

For the transformations in Questions 1-3, what is

$$
    \dfrac{\partial \ \mathbf{T}(d)}{\partial \ d}
$$

that is to say, what is the derivative of the transformation matrix with respect to the transform coordinate $d$?

#### Question 6

Write a Python method that return the partial derivative of the transformation in Question 3 by taking the transform coordinate $d$ as an argument.

#### Question 7

Make a new Python method that returns the spatial velocity $\nu$ of the transformation in Question 3 given the transform coordinate $d$ and velocity $\dot{d}$ as arguments.

#### Question 8

Can you modify your method from Question 6 such that is returns a vector $\mathbf{J}$ that satisfies the following equation?

\begin{align*}
    \Phi \left( \dfrac{\partial \ \mathbf{T}(d)}{\partial \ d} \right) \ \dot{d} &= \mathbf{\nu} \\
    \mathbf{J}(d) \ \dot{d} &= \mathbf{\nu}
\end{align*}

where $\Phi$ is a function that converts the partial derivative in matrix form $\in \mathbb{R}^{4 \times 4}$ to a vector $\mathbf{J} \in \mathbb{R}^6$.

#### Question 9

Research what a Jacobian is and how we may utilise it in robotics. How is a Jacobian related to what we did in Questions 5-8? What system in robotics has several variable transformations and we wish to know the resulting spatial velocity given the transformation coordinate velocities?

<br>

<a id='3'></a>
# 3.0 Transforming Spatial Velocities
---

<img src="./Figures/4/fig3_1a_point.png" width="400">

The Figure above shows two fixed frames and a moving coordinate frame. An observer on the blue fixed frame {$b$} observes an object moving with a spatial velocity $^b\nu$ with respect to frame {$b$}. For an observer on the red frame {$a$}, that frame’s spatial velocity is

\begin{align*}
    {^a\nu} &= 
    \begin{pmatrix}
        {^a\mathbf{R}_b} & \mathbf{0}_{3 \times 3} \\
        \mathbf{0}_{3 \times 3} & {^a\mathbf{R}_b}
    \end{pmatrix}
    {^b\nu} \\
    &= {^a\mathbf{J}_b}({^a\mathbf{T}_b}) \  {^b\nu}
\end{align*}

where ${^a\mathbf{J}_b}(\cdot)$ is a Jacobian or interaction matrix which is a function of the relative orientation of the two frames, and independent of the translation between them. We can investigate this numerically, mirroring the setup of the Figure above.

In [29]:
aTb = sm.SE3.Tx(-2) * sm.SE3.Rz(-np.pi/2) * sm.SE3.Rx(np.pi/2)

If the spatial velocity in frame {$b$} is

In [36]:
bv = np.array([1, 2, 3, 4, 5, 6])

then the spatial velocity in frame {$a$} is

In [40]:
aJb = aTb.jacob()
aJb.shape

av = aJb @ bv

# Display the result
ds.display(ds.Math(f"{{^a\\nu}} = {sym.latex(av)}"))

<IPython.core.display.Math object>

where the jacob method has returned a $6 \times 6$ `NumPy` array. We see that the velocities have been transposed, and sometimes negated due to the different orientation of the frames. The $x$-axis translational and rotational velocities in frame {$b$} have been mapped to the $-y$-axis translational and rotational velocities in frame {$a$}, the $y$-axis velocities have been mapped to the $z$-axis, and the $z$-axis velocities have been mapped to the $-x$-axis.

Next, we consider the case where the two frames are rigidly attached, and moving together, as shown in the Figure below

There is an observer in each frame who is able to estimate their velocity with respect to their own frame. The velocities they observe are related by

\begin{align*}
    {^a\nu} &= 
    \begin{pmatrix}
        {^a\mathbf{R}_b} & [{^a\mathbf{t}_b}] \ {^a\mathbf{R}_b} \\
        \mathbf{0}_{3 \times 3} & {^a\mathbf{R}_b}
    \end{pmatrix}
    {^b\nu} \\
    &= \text{Ad}({^a\mathbf{T}_b}) \ {^b\nu}
\end{align*}

which uses the adjoint matrix of the relative pose and this does depend on the translation between the frames. For example, using the relative pose from the previous example, and with frame {$b$} experiencing pure translation, the velocity experienced by the observer in frame {$a$}

In [42]:
av = aTb.Ad() @ [1, 2, 3, 0, 0, 0]

# Display the result
ds.display(ds.Math(f"{{^a\\nu}} = {sym.latex(av)}"))

<IPython.core.display.Math object>

is also pure translation, but the axes have been transposed due to the relative orientation of the two frames. If the velocity in frame {$b$} is pure angular velocity about the $x$-axis

In [44]:
av = aTb.Ad() @ [0, 0, 0, 1, 0, 0]

# Display the result
ds.display(ds.Math(f"{{^a\\nu}} = {sym.latex(np.round(av, 2))}"))

<IPython.core.display.Math object>

then the observer in frame {$a$} measures that same angular velocity magnitude, but now transposed to be about the $-y$-axis. There is also a translational velocity in the $z$-direction which is due to the origin of fAg following a circular path around the $x$-axis of frame {$b$}. If we combine these translational and rotational velocities in frame {$b$}, then the spatial velocity in frame {$a$} will be

In [45]:
av = aTb.Ad() @ [1, 2, 3, 1, 0, 0]

# Display the result
ds.display(ds.Math(f"{{^a\\nu}} = {sym.latex(np.round(av, 2))}"))

<IPython.core.display.Math object>